In [1]:
import pickle
import numpy as np
import pandas as pd 
import os
import seaborn as sns
from surprise import Dataset
from surprise import Reader
from surprise import SVD
import difflib
import random

In [2]:
user_rating_file_path = "rating_final.csv"
geolocation_file_path = 'geoplaces2.csv'
data = pd.read_csv(user_rating_file_path)
location_data = pd.read_csv(geolocation_file_path)
data = pd.merge(data, location_data[['placeID', 'name']], on='placeID')
data = data[['userID', 'placeID' ,'name', 'rating', 'food_rating', 'service_rating']]
reader = Reader(rating_scale=(0,2))
rating_data = Dataset.load_from_df(data[['userID', 'placeID', 'rating']], reader)
svd = pickle.load(open('Model_Restorants.sav', 'rb'))

In [3]:
def get_rest_id(rest_name, data):
    rest_names = list(data['name'].values)
    closest_names = difflib.get_close_matches(rest_name, rest_names)
    rest_id = data[data['name'] == closest_names[0]]['placeID'].iloc[0]
    return rest_id

def predict_rating(user_id, rest_name, data, model=SVD):
    rest_id = get_rest_id(rest_name, data)
    estimated_ratings = model.predict(uid = user_id, iid = rest_id)
    return estimated_ratings.est

def ten_users():
    print ('10 пользователей из датасета \n')
    for i in range(10):
        random_user_id = np.random.choice(list(np.unique(data['userID'].values)))
        print ('Пользовтель с номером', random_user_id)
        print (pd.Series(data.loc[np.where(data.userID==random_user_id)]['name'].values))
        print ()
    return

def recommend_restaurants(user_id, data=data, model=svd, threshold=1.7):
    recommended_restaurants = {}
    unique_rest_names = list(np.unique(data['name'].values))
    random.shuffle(unique_rest_names)
    for rest_name in unique_rest_names:
        rating = predict_rating(user_id=user_id, rest_name=rest_name, data=data, model=svd)
        if rating > threshold:
            recommended_restaurants[rest_name] = np.round(rating,2)
    print("Генерация рекомендаций ресторана для идентификатора пользователя {} : ".format(user_id))
    restaurant_names = np.array(list(recommended_restaurants.keys())).reshape(-1,1)
    restaurant_ratings = np.array(list(recommended_restaurants.values())).reshape(-1,1)
    results = np.concatenate((restaurant_names, restaurant_ratings), axis=1)
    results_df = pd.DataFrame(results, columns=['Restaurants', 'Rating (0-2)']).sort_values(by='Rating (0-2)', ascending=False)
    return results_df.reset_index().drop('index', axis=1)

In [4]:
ten_users()

10 пользователей из датасета 

Пользовтель с номером U1131
0                la Estrella de Dimas
1    Restaurante y Pescaderia Tampico
2                                 KFC
3                             Chaires
4                       Dominos Pizza
5      Tortas y hamburguesas el gordo
dtype: object

Пользовтель с номером U1052
0                               KFC
1                           Chaires
2                     Dominos Pizza
3    Tortas y hamburguesas el gordo
dtype: object

Пользовтель с номером U1032
0                Tortas Locas Hipocampo
1                 Restaurant la Chalita
2               Restaurante Marisco Sam
3    Cafeteria y Restaurant El Pacifico
4                  Restaurante Tiberius
5           Restaurant Oriental Express
6                            Koye Sushi
7                   Gorditas Doa Gloria
8                         Unicols Pizza
9                       Pizzeria Julios
dtype: object

Пользовтель с номером U1117
0       Cafeteria cenidet
1        El Oc

In [ ]:
print ('Введите номер пользователя, список ресторанов которого к вам ближе всего: ')
random_user_id = (input());
recommend_restaurants(user_id = random_user_id)